In [ ]:
from munkres import Munkres
import copy
import random
import datetime
inf = 999999

In [ ]:
#Если не удовлетворяет правилу треугольника, делаем чтобы удовлетворяло
def floyd_warshall_algorithm(matrix):
    n = len(matrix)
    matrix_new = copy.deepcopy(matrix)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if (matrix_new[i][j] < inf):
                    matrix_new[i][j] = min(matrix_new[i][j], matrix_new[i][k]+matrix_new[k][j])
    return matrix_new

In [ ]:
def hungarian_algorithm(matrix):
    n = len(matrix)
    a = [[inf]*(n+1)]
    for i in range(n):
        a = a + [[inf] + matrix[i]]
    u = [0] * (n+1)
    v = [0] * (n+1)
    p = [0] * (n+1)
    way = [0] * (n+1)
    for i in range(1, n+1):
        p[0] = i
        j_0 = 0
        minv = [inf] * (n+1)
        used = [0] * (n+1)
        while(True):
            used[j_0] = 1
            i_0 = p[j_0]
            delta = inf
            for j in range(1, n+1):
                if (used[j] == 0):
                    cur = a[i_0][j] - u[i_0] - v[j]
                    if (cur < minv[j]):
                        minv[j] = cur
                        way[j] = j_0
                    if (minv[j] < delta):
                        delta = minv[j]
                        j_1 = j
            for j in range(n+1):
                if (used[j] != 0):
                    u[p[j]] += delta
                    v[j] -= delta
                else:
                    minv[j] -= delta
            j_0 = j_1
            if (p[j_0] == 0):
                break
        while(True):
            j_1 = way[j_0]
            p[j_0] = p[j_1]
            j_0 = j_1
            if (j_0 == 0):
                break
    ans = [(-1, -1)] * (n)
    for j in range(1, n+1):
        ans[p[j]-1] = (p[j]-1, j-1)
    return ans

In [ ]:
def Christofides_recurrent(input_matrix, flag_skip_floyd_warshall = False):
    if (flag_skip_floyd_warshall == True):
        matrix = floyd_warshall_algorithm(input_matrix)
    n = len(matrix)          
    idx = hungarian_algorithm(matrix)   #Решаем задачу о назначениях
    way = dict(idx)
    #восстанавливаем циклы
    flags = [0]*n
    cycles = [[]]
    while sum(flags) < n:
        one_cycle = []
        i = flags.index(0)
        while flags[i] == 0:
            flags[i] = 1
            one_cycle.append(i)
            i = way[i]
        cycles.append(one_cycle)
    cycles = cycles[1:]
    if len(cycles) == 1:
        return cycles[0]
    #Объединяем циклы в точку и повторяем
    new_matrix = [[inf] * len(cycles) for i in range(len(cycles))]
    shortcuts = [[(-1, -1)] * len(cycles) for i in range(len(cycles))] 
    for i in range(len(cycles)):
        for j in range(len(cycles)):
            if(i != j):
                for p in range(len(cycles[i])):
                    for q in range(len(cycles[j])):
                        x = cycles[i][p]
                        y = cycles[j][q]
                        if(matrix[x][y] < new_matrix[i][j]):
                            new_matrix[i][j] = matrix[x][y]
                            shortcuts[i][j] = (x, y)
    external_cycle = Christofides_recurrent(new_matrix)
    #Объединяем циклы, выписывая порядки обхода малых циклов по порядку обхода большого
    external_cycle = external_cycle + external_cycle[0:2]
    perfect_cycle = []
    for i in range(1, len(external_cycle)-1):
        sequence_in = shortcuts[external_cycle[i-1]][external_cycle[i]][1]
        next_in = shortcuts[external_cycle[i]][external_cycle[i+1]][1]
        this_cycle = copy.deepcopy(cycles[external_cycle[i]])
        this_cycle = this_cycle + this_cycle[0:2]
        j = this_cycle[1:-1].index(sequence_in)+1
        direct_out = this_cycle[j-1]
        reverse_out = this_cycle[j+1]
        if(matrix[direct_out][next_in]-matrix[direct_out][sequence_in] <= matrix[reverse_out][next_in]-matrix[reverse_out][sequence_in]):
            perfect_cycle = perfect_cycle + this_cycle[j:-1]
            perfect_cycle = perfect_cycle + this_cycle[1:j]
        else:
            part_1 = this_cycle[1:j+1]
            part_1.reverse()
            part_2 = this_cycle[j+1:-1]
            part_2.reverse()
            perfect_cycle = perfect_cycle + part_1 + part_2
    return perfect_cycle

In [ ]:
#2-opt
def algorithm_2opt(matrix, input_sequence):
    sequence = copy.deepcopy(input_sequence)
    flag_is_improved = 1
    while flag_is_improved == 1:
        flag_is_improved = 0
        for i in range(len(sequence)-2):
            for j in range(i+2,min(len(sequence)+i-1, len(sequence))):
                if (matrix[sequence[i]][sequence[i+1]] + matrix[sequence[j]][sequence[(j+1)%len(sequence)]] > matrix[sequence[i]][sequence[j]] + matrix[sequence[i+1]][sequence[(j+1)%len(sequence)]]):
                    rev_part = sequence[i+1:j+1]
                    rev_part.reverse()
                    sequence = sequence[:i+1] + rev_part + sequence[j+1:]
                    flag_is_improved = 1
    return sequence

In [ ]:
infile = open('fqm5087.tsp.txt', 'r')
Dimension = 5087

# Read node list
nodelist = []
N = int(Dimension)
for i in range(0, int(Dimension)):
    x,y = infile.readline().strip().split()[1:]
    nodelist.append([float(x), float(y)])

# Close input file
infile.close()

In [ ]:
from math import *
n = len(nodelist)
input_matrix = [[inf] * n for i in range(n)]
for i in range(0,n-1):
    for j in range(i+1, n):
        a = int(sqrt((nodelist[i][0] - nodelist[j][0])*(nodelist[i][0] - nodelist[j][0]) + (nodelist[i][1] - nodelist[j][1])*(nodelist[i][1] - nodelist[j][1])))
        input_matrix[i][j] = a
        input_matrix[j][i] = a

In [ ]:
print(datetime.datetime.now())
seq = Christofides_recurrent(input_matrix, flag_skip_floyd_warshall = True)
print(datetime.datetime.now())
algorithm_2opt(input_matrix, seq)
print(datetime.datetime.now())